In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence impofrt pad_sequences
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam

In [8]:
train = pd.read_json('train.json')

train['ingredients_str'] = train['ingredients'].apply(lambda x: ' '.join(x))

le = LabelEncoder()
train['cuisine'] = le.fit_transform(train['cuisine'])

In [9]:
X_train, X_test, y_train, y_test = train_test_split(train['ingredients_str'], train['cuisine'], test_size=0.2, random_state=42)

tokenizer = Tokenizer(num_words=20000, oov_token='<OOV>')
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

max_len = 100
X_train_padded = pad_sequences(X_train_seq, maxlen=max_len, padding='post')
X_test_padded = pad_sequences(X_test_seq, maxlen=max_len, padding='post')

In [13]:
embeddings_index = {}
with open('glove.6B.100d.txt', 'r', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        try:
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
        except ValueError:
            print(f"Skipping line with formatting issue: {line[:50]}...")

embedding_dim = 100
word_index = tokenizer.word_index
num_words = min(20000, len(word_index) + 1)

embedding_matrix = np.zeros((num_words, embedding_dim))
for word, i in word_index.items():
    if i >= 20000:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

Skipping line with formatting issue: standard -0.13416 0.47697 0.45242 0.2767 -0.25912 ...


In [14]:
model = Sequential()
model.add(Embedding(input_dim=num_words,
                    output_dim=embedding_dim,
                    input_length=max_len,
                    weights=[embedding_matrix],
                    trainable=False))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Dropout(0.5))
model.add(Bidirectional(LSTM(64)))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dense(len(le.classes_), activation='softmax'))

In [ ]:
optimizer = Adam(learning_rate=0.001)
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

history = model.fit(X_train_padded, y_train, epochs=20, validation_data=(X_test_padded, y_test), batch_size=64)

model.save('improved_cuisine_classification_model.h5')

loss, accuracy = model.evaluate(X_test_padded, y_test)
print(f'Model Accuracy: {accuracy * 100:.2f}%')

Epoch 1/20
498/498 [==============================] - 19s 25ms/step - loss: 2.1990 - accuracy: 0.3552 - val_loss: 1.8896 - val_accuracy: 0.4430
Epoch 2/20
498/498 [==============================] - 12s 24ms/step - loss: 1.8736 - accuracy: 0.4471 - val_loss: 1.7883 - val_accuracy: 0.4669
Epoch 3/20
498/498 [==============================] - 12s 24ms/step - loss: 1.7927 - accuracy: 0.4676 - val_loss: 1.7588 - val_accuracy: 0.4724
Epoch 4/20
433/498 [=========================>....] - ETA: 1s - loss: 1.7319 - accuracy: 0.4853